<a href="https://colab.research.google.com/github/manuel-alvarez/scheduling/blob/master/poc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Schedule Generator with Genetic Algorithms

## POC

In [0]:
import numpy

In [0]:
POPULATION_SIZE = 11
INDIVIDUAL_SIZE = 10
RESOURCES = 5
SURVIVAL_RATE = .6

In [0]:
def initialize_population(population_size, individual_size, resources_size):
  population = None
  for i in range(population_size):
    if population is None:
      population = numpy.array([numpy.random.randint(resources_size + 1, size=individual_size)])
    else:
      population = numpy.append(population, [numpy.random.randint(resources_size + 1, size=individual_size)], axis=0)
  return population

In [0]:
def get_fitness(the_try):
  unique, counts = numpy.unique(the_try, return_counts=True)
  return sum(item for item in counts if item > 1)

In [0]:
def get_selection(population):
  return numpy.array(sorted([[get_fitness(the_try), the_try] for the_try in population], key=lambda x:x[0])[:int(numpy.ceil(POPULATION_SIZE * SURVIVAL_RATE))])

In [0]:
def breed(selection):
  population = numpy.array([the_try for the_try in selection])
  i = 0
  while len(population) < POPULATION_SIZE:
    parents = population[i:2]
    numpy.random.shuffle(parents)
    slicery = numpy.random.randint(INDIVIDUAL_SIZE) 
    population = numpy.append(population, numpy.array([numpy.append(parents[0][slicery:], parents[1][:slicery], axis=0)]), axis=0)
  return population

In [0]:
def mutate(selection):
  pass

In [74]:
population = initialize_population(POPULATION_SIZE, INDIVIDUAL_SIZE, RESOURCES)
selection = get_selection(population)
population = breed(selection[:,1])
print(population)

[1 3 3 2 5 5 1 2 2 0]
[1 3 3 2 5 5 1 2 2 0]
[5 3 0 2 4 3 1 3 3 2]
[5 3 0 2 4 3 1 3 3 2]
[3 3 2 5 5 1 2 2 0 4]
[3 3 2 5 5 1 2 2 0 4]
[2 3 0 5 3 0 2 4 3 1]
[2 3 0 5 3 0 2 4 3 1]
[[2 5 5 1 2 2 0 4 2 3]
 [0 5 3 0 2 4 3 1 3 3]
 [1 2 4 3 0 4 5 2 0 4]
 [0 3 1 1 2 2 4 3 1 3]
 [5 4 0 2 3 4 2 2 2 0]
 [4 0 4 3 3 1 2 1 0 5]
 [3 5 4 0 2 5 0 5 3 5]
 [1 3 3 2 5 5 1 2 2 0]
 [5 3 0 2 4 3 1 3 3 2]
 [3 3 2 5 5 1 2 2 0 4]
 [2 3 0 5 3 0 2 4 3 1]]
